### read segmentation data 

In [1]:
import cellshape_helper as helper
from tifffile import imread
import napari
from skimage.measure import marching_cubes, regionprops
import trimesh
import numpy as np

In [4]:
membrane_dir = 'data/test_membrane.tif'
nuclei_dir = 'data/test_nuclei.tif'
membrane_raw_dir = 'data/membrane_raw.tif'
nuclei_raw_dir = 'data/nuclei_raw.tif'

membrane_seg = imread(membrane_dir)
nuclei_seg = imread(nuclei_dir)
membrane_raw = imread(membrane_raw_dir)
nuclei_raw = imread(nuclei_raw_dir)

### `tif_to_mesh` rewrite the function without saving

In [5]:
# marching_cubes require 3D input
vertices, faces, normals, values = marching_cubes(x)
mesh_obj = trimesh.Trimesh(
            vertices=vertices, faces=faces, process=False
        )
mesh_obj.export('data/test_nuclei_x.off')

In [10]:
from cellshape_helper.vendor.pytorch_geometric_files import read_off, sample_points
# dict pos, face
data = read_off('data/test_nuclei_x.off')
# change to .numpy() 
num_points = 1024
points = sample_points(data=data, num=num_points).numpy()

viewer = napari.Viewer()
viewer.add_image(nuclei_raw[0,6:14,1024:1040,895:910])
viewer.add_labels(x)

# Add the 3D mesh to the viewer
viewer.add_surface((vertices, faces, values), name="Marching Cubes Mesh")

# Add the sampled points to the viewer
viewer.add_points(points, size=1, name="Sampled Points")

### Creat single cell box

In [5]:
from skimage import measure
from tifffile import imwrite
from skimage import measure
import os
from tqdm import tqdm


def get_max_dims(seg_img):
    max_dims = np.array([0, 0, 0])
    # Determine the dimensions of the largest bounding box
    for region in measure.regionprops(seg_img):
        minz, miny, minx, maxz, maxy, maxx = region.bbox
        dims = np.array([maxz-minz, maxy-miny, maxx-minx])
        max_dims = np.maximum(max_dims, dims)
    
    return max_dims


seg_img = nuclei_seg[0]
max_dims = get_max_dims(seg_img)

# Add a margin to max_dims
margin = np.array([5, 10, 10])  # adjust the values as needed
max_dims_with_margin = max_dims + margin

In [6]:
def save_individual_label(seg_img, save_singlecell_dir, max_dims_with_margin,t):
    regions = measure.regionprops(seg_img)
    num_cells = len(regions)
    all_cells = np.zeros((num_cells, *max_dims_with_margin), dtype=seg_img.dtype)

    if not os.path.exists(save_singlecell_dir):
        os.makedirs(save_singlecell_dir)

    # Wrap the loop with tqdm to create a progress bar
    for i, region in tqdm(enumerate(regions), total=num_cells, desc="Processing Cells"):
        minz, miny, minx, maxz, maxy, maxx = region.bbox
        cropped_image = seg_img[minz:maxz, miny:maxy, minx:maxx]
        label_mask = (cropped_image == region.label)

        padding = [
            ((max_dims_with_margin[0] - label_mask.shape[0]) // 2,
             max_dims_with_margin[0] - label_mask.shape[0] - ((max_dims_with_margin[0] - label_mask.shape[0]) // 2)),
            ((max_dims_with_margin[1] - label_mask.shape[1]) // 2,
             max_dims_with_margin[1] - label_mask.shape[1] - ((max_dims_with_margin[1] - label_mask.shape[1]) // 2)),
            ((max_dims_with_margin[2] - label_mask.shape[2]) // 2,
             max_dims_with_margin[2] - label_mask.shape[2] - ((max_dims_with_margin[2] - label_mask.shape[2]) // 2))
        ]

        padded_image = np.pad(label_mask, padding, mode='constant', constant_values=0)
        # Save the padded image with the label as the filename
        imwrite(f"{save_singlecell_dir}/{str(t)+region.label}.tif", padded_image)
        all_cells[i] = padded_image

    return all_cells  # This line is now correctly placed outside of the for loop.



save_singlecell_dir = 'data/singlecell'
all_cells = save_individual_label(seg_img, save_singlecell_dir, max_dims_with_margin)

Processing Cells: 100%|██████████| 921/921 [00:02<00:00, 357.84it/s]


In [11]:
viewer = napari.Viewer()
viewer.add_labels(all_cells)

<Labels layer 'all_cells' at 0x2cfcedef880>

### cellshape

#### read from crop pointcloud 

In [7]:
PATH_TO_TIF_FILES = "data/singlecell"
PATH_TO_SAVE_MESH = "data/mesh/"
PATH_TO_SAVE_PC = "data/pointcloud/"
NUM_POINTS = 2048

helper.tif_to_pc_directory(PATH_TO_TIF_FILES, 
                           PATH_TO_SAVE_MESH, 
                           PATH_TO_SAVE_PC, 
                           NUM_POINTS)

100%|██████████| 921/921 [00:11<00:00, 77.86it/s]


In [8]:
import torch
from cellshape_cloud import CloudAutoEncoder

model = CloudAutoEncoder(num_features=128, 
                         k=20,
                         encoder_type="dgcnn",
                         decoder_type="foldingnet")



# model is not trained so far
points = torch.randn(1, 2048, 3)
recon, features = model(points)

In [3]:

# Convert tensor to numpy array
recon_np = recon.detach().cpu().numpy()
viewer = napari.Viewer()
viewer.add_points(points.numpy()[0], size=1, name="Test Points")
viewer.add_points(recon_np[0], size=1, name="recon Points")

<Points layer 'recon Points' at 0x129c6127a00>

#### To train an autoencoder on a set of point clouds created using cellshape-helper:

In [9]:
import torch
from torch.utils.data import DataLoader

import cellshape_cloud as cloud
from cellshape_cloud.vendor.chamfer_distance import ChamferLoss

input_dir = "data/pointcloud/"
batch_size = 16
learning_rate = 0.0001
num_epochs = 100
output_dir = "model_100epoch/model_output/"

model = cloud.CloudAutoEncoder(num_features=128, 
                         k=40,
                         encoder_type="dgcnn",
                         decoder_type="foldingnet")


dataset = cloud.PointCloudDataset(input_dir)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:
"""
according to cellshape_cloud/pointcloud_dataset.py class PointCloudDataset(Dataset)
each data point in your dataset is a tuple of four elements: (point_cloud, 0, 0, 0)

data_iter = iter(dataloader)
first_batch = next(data_iter)
first_batch_point_clouds = first_batch[0]

first_batch_point_clouds shape is (16,2048,3)
first_batch[1], first_batch[2], first_batch[4] is (16,) the value is 0


from cellshape_cloud.vendor import get_graph_feature
data_iter = iter(dataloader)
point_clouds = next(data_iter)
x = point_clouds[0]
x = x.transpose(2, 1)
batch_size = x.size(0)
graph_feature = get_graph_feature(x, k=40)
knn_graph = graph_feature.numpy()
knn_graph has shape (16,6,2048,20)
"""

In [10]:

criterion = ChamferLoss()

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate * 16 / batch_size,
    betas=(0.9, 0.999),
    weight_decay=1e-6,
)

#passing correct logging_infoto `clou_train` by
# Define paths and experiment name
name_logging = output_dir + "logging/directory"
name_model = output_dir + "save/model.pth"
name_writer = output_dir + "tensorboard/logs"
name = "my_experiment"

# Make sure directories exist
os.makedirs(name_logging, exist_ok=True)
os.makedirs(os.path.dirname(name_model), exist_ok=True)  # Since name_model includes the filename
os.makedirs(name_writer, exist_ok=True)

# Create the logging_info tuple
logging_info = (name_logging, name_model, name_writer, name)

cloud.train(model, dataloader, num_epochs, criterion, optimizer, logging_info)


Epoch 0: 100%|██████████| 58/58 [00:24<00:00,  2.34batch/s, loss=366]


Saving model to model_100epoch/model_output/save/model.pth with loss = 329.86309101207047.
Finished epoch 0 with loss=329.86309101207047.


Epoch 1: 100%|██████████| 58/58 [00:23<00:00,  2.48batch/s, loss=274]


Saving model to model_100epoch/model_output/save/model.pth with loss = 313.922493419428.
Finished epoch 1 with loss=313.922493419428.


Epoch 2: 100%|██████████| 58/58 [00:23<00:00,  2.47batch/s, loss=213]


...

Epoch 98: 100%|██████████| 58/58 [00:23<00:00,  2.46batch/s, loss=1.91]


Saving model to model_100epoch/model_output/save/model.pth with loss = 2.154494004916414.
Finished epoch 98 with loss=2.154494004916414.


Epoch 99: 100%|██████████| 58/58 [00:23<00:00,  2.47batch/s, loss=1.84]

Saving model to model_100epoch/model_output/save/model.pth with loss = 2.0414001083008633.
Finished epoch 99 with loss=2.0414001083008633.
Finished training 100 epochs.


(CloudAutoEncoder(
   (encoder): DGCNNEncoder(
     (conv1): Sequential(
       (0): Conv2d(6, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): LeakyReLU(negative_slope=0.2)
     )
     (conv2): Sequential(
       (0): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): LeakyReLU(negative_slope=0.2)
     )
     (conv3): Sequential(
       (0): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): LeakyReLU(negative_slope=0.2)
     )
     (conv4): Sequential(
       (0): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): LeakyReLU(negative_s